In [1]:
import sympy as sym
import pandas as pd
from math import ceil, floor, log2
import enum

In [2]:
class CoeffType(enum.Enum):
    Type1Pad = 1
    Type2 = 2
    Type3Pad = 3
    Type4 = 4
    NonLinPhase = 5
    HalfBand = 6

def hffa(p, ws):
    if p==1:
        return [ws]
    return hffa(p//2, ws[0::2]) + \
           hffa(p//2, [a+b for (a,b) in zip(ws[0::2],ws[1::2])]) + \
           hffa(p//2, ws[1::2])

def hpoly(p, ws):
    return [ws for _ in range(p)]

def eqTerms(x, y):
    return 0 == sym.simplify(x-y)

# NOTE: We aren't checking if one term is a bitshifted version of an existing one... can we prove this will not happen?
def uniqAbsTerms(xs):
    uniqs = []
    for x in xs:
        if not ( any([eqTerms(sym.Abs(x),u) for u in uniqs]) or \
                 eqTerms(x,0) ):
            uniqs.append(sym.Abs(x))
    return uniqs

def countMults(h):
    return sum([len(ws) for ws in h])

_nonlinPhase = lambda n : [sym.Symbol('w'+str(i)) for i in range(n)]

_type2 = lambda n : [sym.Symbol('w'+str(i)) for i in range(n//2)] + \
                    [sym.Symbol('w'+str(i)) for i in range(n//2-1,-1,-1)]

_type4 = lambda n : [sym.Symbol('w'+str(i)) for i in range(n//2)] + \
                    [(-1)*sym.Symbol('w'+str(i)) for i in range(n//2-1,-1,-1)]

_type1Padded = lambda n : [sym.Symbol('w'+str(i)) for i in range(n//2 - 1)] + \
                          [sym.Symbol('w'+str(n//2-1))] + \
                          [sym.Symbol('w'+str(i)) for i in range(n//2-1-1,-1,-1)] + \
                          [0]

_type3Padded = lambda n : [sym.Symbol('w'+str(i)) for i in range(n//2 - 1)] + \
                          [sym.Symbol('w'+str(n//2-1))] + \
                          [(-1)*sym.Symbol('w'+str(i)) for i in range(n//2-1-1,-1,-1)] + \
                          [0]

def _halfBand(n):
    ws = [0 for _ in range(n)]
    if n//2 % 2 == 1:
        ws[::2] = _type1Padded(n//2+1)[:-1]
    else:
        ws[::2] = _type1Padded(n//2)
    return ws

def genCoeffs(coeffType, n):
    if coeffType == CoeffType.Type1Pad:
        return _type1Padded(n)
    if coeffType == CoeffType.Type2:
        return _type2(n)
    if coeffType == CoeffType.Type3Pad:
        return _type3Padded(n)
    if coeffType == CoeffType.Type4:
        return _type4(n)
    if coeffType == CoeffType.NonLinPhase:
        return _nonlinPhase(n)
    if coeffType == CoeffType.HalfBand:
        return _halfBand(n)

In [3]:
def ffa_hnlp(par, n):
    p = int(log2(par))
    return n*3**(p)

def ffa_htype2(par, n):
    p = int(log2(par))
    return ceil(n/2.0)+2*n*sum([3**i for i in range(0,p)])

def ffa_htype4(par, n):
    p = int(log2(par))
    return floor(n/2.0)+2*n*sum([3**i for i in range(0,p)])

def ffa_htype1(par, n):
    p = int(log2(par))
    return n*(2 + sum([3**k for k in range(1,p)]) + 2*sum([sum ([3**j for j in range(0,i+1)]) for i in range(0,p-1)])) + (p-1)*(ceil(n/2.0))

def ffa_htype3(par, n):
    p = int(log2(par))
    return n*(2 + sum([3**k for k in range(1,p)]) + 2*sum([sum ([3**j for j in range(0,i+1)]) for i in range(0,p-1)])) + (p-1)*(ceil(n/2.0))

def ffa_hhb(par, n):
    p = int(log2(par))
    if p == 1:
        return 2*ceil(n/2.0)
    return 2 * ffa_htype1(2**(p-1), n)

def ffa_eqns(ctype, par, n):
    if ctype == CoeffType.Type1Pad:
        return ffa_htype1(par, n)
    if ctype == CoeffType.Type2:
        return ffa_htype2(par, n)
    if ctype == CoeffType.Type3Pad:
        return ffa_htype3(par, n)
    if ctype == CoeffType.Type4:
        return ffa_htype4(par, n)
    if ctype == CoeffType.NonLinPhase:
        return ffa_hnlp(par, n)
    if ctype == CoeffType.HalfBand:
        return ffa_hhb(par,n)

def poly_eqns(ctype, par, n):
    taps = par*n
    if ctype == CoeffType.Type1Pad:
        return par*ceil(taps/2.0)
    if ctype == CoeffType.Type2:
        return par*floor(taps/2.0)
    if ctype == CoeffType.Type3Pad:
        return par*ceil(taps/2.0)
    if ctype == CoeffType.Type4:
        return par*floor(taps/2.0)
    if ctype == CoeffType.NonLinPhase:
        return par*taps
    if ctype == CoeffType.HalfBand:
        return par*ceil(taps/4.0)

In [4]:
frame = pd.DataFrame(columns=['FIR', 'CoeffType', 'Parallelism', 'N', 'SubfiltN', 'Mults'])

for p in [2,4,8,16]:
    print(f'Running p{p}')
    for nByP in [1,2,3,4]:
        print(f'Running nByP{nByP}')
        for coeffType in CoeffType:
            ws = genCoeffs(coeffType, p*nByP)
            
            mults = ffa_eqns(coeffType, p, nByP)
            frame = frame.append(dict(FIR='FFAEqn', CoeffType=coeffType.name, Parallelism=p, N=(p*nByP), SubfiltN=nByP, Mults=mults, MultsPerChannel=mults/p), ignore_index=True)
            
            mults = countMults([uniqAbsTerms(h) for h in hffa(p, ws)])
            frame = frame.append(dict(FIR='FFA'  , CoeffType=coeffType.name, Parallelism=p, N=(p*nByP), SubfiltN=nByP, Mults=mults, MultsPerChannel=mults/p), ignore_index=True)
            
            mults = poly_eqns(coeffType, p, nByP)
            frame = frame.append(dict(FIR='PolyEqn', CoeffType=coeffType.name, Parallelism=p, N=(p*nByP), SubfiltN=nByP, Mults=mults, MultsPerChannel=mults/p), ignore_index=True)
            
            mults = countMults([uniqAbsTerms(h) for h in hpoly(p, ws)])
            frame = frame.append(dict(FIR='Poly' , CoeffType=coeffType.name, Parallelism=p, N=(p*nByP), SubfiltN=nByP, Mults=mults, MultsPerChannel=mults/p), ignore_index=True)

Running p2
Running nByP1
Running nByP2
Running nByP3
Running nByP4
Running p4
Running nByP1
Running nByP2
Running nByP3
Running nByP4
Running p8
Running nByP1
Running nByP2
Running nByP3
Running nByP4
Running p16
Running nByP1
Running nByP2
Running nByP3
Running nByP4


In [13]:
frame = pd.DataFrame(columns=['FIR', 'CoeffType', 'Parallelism', 'N', 'SubfiltN', 'Mults'])

for p in [2,4,8,16]:
    print(f'Running p{p}')
    for nByP in [1,2,3,4,5]:
        print(f'Running nByP{nByP}')
        for coeffType in CoeffType:
            ws = genCoeffs(coeffType, p*nByP)
            
            mults = ffa_eqns(coeffType, p, nByP)
            frame = frame.append(dict(FIR='FFA', CoeffType=coeffType.name, Parallelism=p, N=(p*nByP), SubfiltN=nByP, Mults=mults, MultsPerChannel=mults/p), ignore_index=True)
             
            mults = poly_eqns(coeffType, p, nByP)
            frame = frame.append(dict(FIR='Poly', CoeffType=coeffType.name, Parallelism=p, N=(p*nByP), SubfiltN=nByP, Mults=mults, MultsPerChannel=mults/p), ignore_index=True)

Running p2
Running nByP1
Running nByP2
Running nByP3
Running nByP4
Running nByP5
Running p4
Running nByP1
Running nByP2
Running nByP3
Running nByP4
Running nByP5
Running p8
Running nByP1
Running nByP2
Running nByP3
Running nByP4
Running nByP5
Running p16
Running nByP1
Running nByP2
Running nByP3
Running nByP4
Running nByP5


In [6]:
frame.to_csv('outputs/coeffs.csv', index = False, header=True)

In [4]:
frame = pd.read_csv("outputs/coeffs.csv")

In [5]:
import plotly.express as px
fig = px.line(frame[frame['CoeffType']!='Type3Pad'][frame['CoeffType']!='Type4'],
        x='SubfiltN',y='MultsPerChannel', color='FIR', facet_row='Parallelism', facet_col='CoeffType',  category_orders={"CoeffType": ['NonLinPhase','Type1Pad','Type2','HalfBand']}, height=1000)

<ipython-input-5-491b306b6fe1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fig = px.line(frame[frame['CoeffType']!='Type3Pad'][frame['CoeffType']!='Type4'],


In [6]:
frame

,FIR,CoeffType,Parallelism,N,SubfiltN,Mults,MultsPerChannel
0,FFAEqn,Type1Pad,2,2,1,2,1.0
1,FFA,Type1Pad,2,2,1,2,1.0
2,PolyEqn,Type1Pad,2,2,1,2,1.0
3,Poly,Type1Pad,2,2,1,2,1.0
4,FFAEqn,Type2,2,2,1,3,1.5
...,...,...,...,...,...,...,...
379,Poly,NonLinPhase,16,64,4,1024,64.0
380,FFAEqn,HalfBand,16,64,4,200,12.5
381,FFA,HalfBand,16,64,4,200,12.5
382,PolyEqn,HalfBand,16,64,4,256,16.0


In [11]:
for p in [2,4,8,16]:
    for coeffType in CoeffType:
        frame = pd.DataFrame(columns=['N', 'FfaMults', 'PolyMults'])
        frame = frame.append(dict(N=0, FfaMults=0, PolyMults=0), ignore_index=True)
        for nByP in [1,2,3,4,5]:
            ws = genCoeffs(coeffType, p*nByP)
            
            mults = ffa_eqns(coeffType, p, nByP)
            frame = frame.append(dict(N=(p*nByP), FfaMults=ffa_eqns(coeffType, p, nByP), PolyMults=poly_eqns(coeffType, p, nByP)), ignore_index=True)
            
        frame.to_csv(f'outputs/coeffs_p{p}_{coeffType.name}.csv', index = False, header=True)

In [12]:
frame

,N,FfaMults,PolyMults
0,0,0,0
1,16,52,64
2,32,100,128
3,48,152,192
4,64,200,256
5,80,252,320
